In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [2]:
import numpy as np
import os

np.random.seed(42)

DATA_PATH = 'dataset'

In [3]:
from scipy.io import arff
import pandas as pd

def load_pulsar_csv(path = DATA_PATH):
    csv_path = os.path.join(path, 'HTRU_2.csv')
    return np.loadtxt(csv_path, delimiter=',', dtype=np.float32)

def load_pulsar_arff(path = DATA_PATH):
    arff_path = os.path.join(path, 'HTRU_2.arff')
    return arff.loadarff(arff_path)

In [4]:
pulsars = load_pulsar_csv()

In [5]:
import numpy as np

def split_train_dataset(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(test_ratio * len(data))
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data[train_indices,:], data[test_indices,:]

# Use hash of identifier to decide if instance goes into train or test set

In [6]:
train_set, test_set = split_train_dataset(pulsars, 0.2)

In [7]:
X_train, Y_train = train_set[:, :-1], train_set[:, -1]
X_test, Y_test = test_set[:, :-1], test_set[:, -1]

In [20]:
# Create Model
model = Sequential()

input_dimension = np.size(X_train, axis=1)
model.add(Dense(32, input_dim=input_dimension, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [21]:
# Compile Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
# Fit the Model
model.fit(X_train, Y_train, epochs=50, batch_size=10, validation_split=0.2)

Train on 11455 samples, validate on 2864 samples
Epoch 1/50
11455/11455 [==============================] - 3s 300us/step - loss: 0.0729 - acc: 0.9800 - val_loss: 0.0673 - val_acc: 0.9822
Epoch 2/50
11455/11455 [==============================] - 3s 300us/step - loss: 0.0724 - acc: 0.9798 - val_loss: 0.0647 - val_acc: 0.9797
Epoch 3/50
11455/11455 [==============================] - 3s 300us/step - loss: 0.0725 - acc: 0.9795 - val_loss: 0.0654 - val_acc: 0.9794
Epoch 4/50
11455/11455 [==============================] - 3s 305us/step - loss: 0.0718 - acc: 0.9803 - val_loss: 0.0664 - val_acc: 0.9791
Epoch 5/50
11455/11455 [==============================] - 3s 300us/step - loss: 0.0723 - acc: 0.9790 - val_loss: 0.0740 - val_acc: 0.9784
Epoch 6/50
11455/11455 [==============================] - 4s 327us/step - loss: 0.0730 - acc: 0.9793 - val_loss: 0.0661 - val_acc: 0.9780
Epoch 7/50
11455/11455 [==============================] - 3s 295us/step - loss: 0.0721 - acc: 0.9795 - val_loss: 0.0693 - v

In [62]:
# Evaluate the Model
scores = model.evaluate(X_test, Y_test)

3579/3579 [==============================] - 0s 33us/step


In [63]:
print("Accuracy:", scores[1] * 100)

Accuracy: 98.1559094769


In [75]:
# Save Model
import os

MODELS_DIR = "models"

def save_model(model, name, acc=None):
    name += str(model.input.shape[1])
    for layer in model.layers:
        name += "-" + str(layer.output.shape[1])
    
    name += "_" + (("%.2f" % acc) if acc is not None else "")
    path = os.path.join(MODELS_DIR, name + ".h5")
    model.save(path)

In [76]:
save_model(model, 'pulsar', scores[1] * 100)